In [2]:
!pip3 install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 23.0 MB/s eta 0:00:00m eta 0:00:010:0101
  Using cached scipy-1.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (41.2 MB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)


In [18]:
!pip install bert-score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 KB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 31.4 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached contourpy-1.3.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (324 kB)
  Using cached kiwisolver-1.4.7-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.6 MB)
  Using cached pyparsing-3.2.0-py3-none-any.whl (106 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 10.2 MB/s eta 0:00:00m eta 0:00:01:01m
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)


In [10]:
import pandas as pd
import openai
import os
from dotenv import load_dotenv
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import T5Tokenizer, T5ForConditionalGeneration
import nltk
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [12]:
# Step 1: Load the data
df = pd.read_csv('summery_training.csv', encoding='latin1')
print(df.head())

# Extract original policy descriptions
original_docs = df['policy_description'].tolist()

# Extract GPT-4o-mini summaries
gpt_summaries = df['Summary'].tolist()

                    country  \
0      United Arab Emirates   
1                Kazakhstan   
2              Saudi Arabia   
3  United States of America   
4                   Finland   

                                  policy_description  \
0  The National Hydrogen Strategy 2050 aims to su...   
1  The code regulates activities which may have a...   
2  Saudi Arabia aims to increase the share of nat...   
3              Focused on better lighting solutions.   
4  Investment proposal for areas with coal-fired ...   

                                             Summary  \
0  The National Hydrogen Strategy 2050 aims to po...   
1  A code requiring the largest polluters to adop...   
2  Saudi Arabia aims for 50% of electricity from ...   
3  Focus on enhancing lighting solutions for ener...   
4  Proposal for investing in regions with coal pl...   

                             facebook/bart-large-cnn  \
0  The National Hydrogen Strategy 2050 aims to su...   
1  The code regulates activ

In [13]:
summarizer_model = AutoModelForSeq2SeqLM.from_pretrained("./flan-t5-policy-finetune")
summarizer_tokenizer = AutoTokenizer.from_pretrained("./flan-t5-policy-finetune")

In [14]:
# Function to summarize documents using your local model
def summarize_documents(docs, max_length=200):  # Increase max_length for more detailed summaries
    summaries = []
    for doc in docs:
        input_text = "summarize: " + doc
        inputs = summarizer_tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
        summary_ids = summarizer_model.generate(
            inputs,
            max_length=max_length,
            min_length=100,  # Increase min_length for more detailed summaries
            length_penalty=1.5,  # Adjust length penalty to encourage longer summaries
            num_beams=6,  # Increase num_beams for better quality
            early_stopping=True
        )
        summary = summarizer_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(summary)
    return summaries

# Generate summaries using your local model
local_summaries = summarize_documents(original_docs)

In [15]:
# Function to calculate cosine similarity between two lists of summaries
def calculate_cosine_similarity(list1, list2):
    vectorizer = TfidfVectorizer().fit_transform(list1 + list2)
    vectors = vectorizer.toarray()
    cosine_sim = cosine_similarity(vectors[:len(list1)], vectors[len(list1):])
    return cosine_sim.diagonal().mean()

# Calculate and print cosine similarity between GPT-4o-mini summaries and local model summaries
cosine_sim = calculate_cosine_similarity(gpt_summaries, local_summaries)
print(f'Cosine Similarity between GPT-4o-mini and Local Model Summaries: {cosine_sim:.4f}')

# Calculate and print the Mean Squared Error (MSE) between GPT-4o-mini and Local Model summaries length
mse_length = mean_squared_error([len(s) for s in gpt_summaries], [len(s) for s in local_summaries])
print(f'Mean Squared Error of summary lengths: {mse_length:.4f}')

Cosine Similarity between GPT-4o-mini and Local Model Summaries: 0.3637
Mean Squared Error of summary lengths: 180642.0000


In [16]:
# Function to calculate BLEU score
def calculate_bleu_score(reference_summaries, generated_summaries):
    bleu_scores = []
    for reference, generated in zip(reference_summaries, generated_summaries):
        reference_tokens = [reference.split()]
        generated_tokens = generated.split()
        score = sentence_bleu(reference_tokens, generated_tokens)
        bleu_scores.append(score)
    return sum(bleu_scores) / len(bleu_scores)

# Calculate and print BLEU score
bleu_score = calculate_bleu_score(gpt_summaries, local_summaries)
print(f'BLEU Score between GPT-4o-mini and Local Model Summaries: {bleu_score:.4f}')

BLEU Score between GPT-4o-mini and Local Model Summaries: 0.0203


/home/ubuntu/myenv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/ubuntu/myenv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/ubuntu/myenv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunc

In [17]:
# Calculate ROUGE scores
def calculate_rouge_scores(gpt_summaries, local_summaries):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
    
    for gpt_summary, local_summary in zip(gpt_summaries, local_summaries):
        score = scorer.score(local_summary, gpt_summary)
        for key in scores.keys():
            scores[key].append(score[key].fmeasure)
    
    avg_scores = {key: sum(values) / len(values) for key, values in scores.items()}
    return avg_scores

# Calculate and print ROUGE scores between GPT-4o-mini summaries and local model summaries
rouge_scores = calculate_rouge_scores(gpt_summaries, local_summaries)
print(f'ROUGE Scores:')
for metric, score in rouge_scores.items():
    print(f'{metric.upper()}: {score:.4f}')

ROUGE Scores:
ROUGE1: 0.2256
ROUGE2: 0.1071
ROUGEL: 0.1877


In [19]:
from bert_score import score

# Function to calculate BERTScore
def calculate_bert_score(reference_summaries, candidate_summaries, model_name='bert-base-uncased'):
    P, R, F1 = score(candidate_summaries, reference_summaries, model_type=model_name, lang="en")
    avg_f1 = F1.mean().item()
    return avg_f1, P.mean().item(), R.mean().item()

# Calculate BERTScore between GPT-4o-mini summaries and Local Model summaries
bert_avg_f1, bert_precision, bert_recall = calculate_bert_score(gpt_summaries, local_summaries)

# Print the BERTScore results
print(f"BERTScore Results:")
print(f"Precision: {bert_precision:.4f}")
print(f"Recall: {bert_recall:.4f}")
print(f"F1 Score: {bert_avg_f1:.4f}")


BERTScore Results:
Precision: 0.5129
Recall: 0.7018
F1 Score: 0.5874
